In [1]:
import os
from dotenv import load_dotenv
import dtlpy as dl
from rfdetr.util.coco_classes import COCO_CLASSES



In [4]:
use_rc_env = False
if use_rc_env:
    dl.setenv('rc')
else:
    dl.setenv('prod')
if dl.token_expired():
    dl.login()


In [5]:
if use_rc_env:
    project = dl.projects.get(project_name='HusamDimo')
else:
    #project = dl.projects.get(project_name='ShadiDemo')
    project = dl.projects.get(project_name='IPM development')

In [6]:
models = project.models.list().print()



+----+--------------------------+----------------------+---------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
# Publish your app
dpk = project.dpks.publish()
# Install or update the application
try:
    app = project.apps.get(app_name=dpk.display_name)
    app.dpk_version = dpk.version
    app.update()
except dl.exceptions.NotFound:
    print("installing ...")
    app = project.apps.install(dpk=dpk)

Command Progress: 100%|██████████| 100/100 [00:06<00:00, 15.64it/s]


In [6]:
model = project.models.get(model_name = 'rf-detr')

KeyboardInterrupt: 

In [65]:
model.open_in_web()

set the lables from COCO_CLASSES

In [7]:
model.labels = list(COCO_CLASSES.values())

In [ ]:
len(COCO_CLASSES.values())


In [56]:
artifact = model.artifacts.upload(filepath='rf-detr-base-coco.pth')


Upload Items: 355MB [00:42, 8.78MB/s]                           
Iterate Entity: 100%|██████████| 1/1 [00:00<00:00,  6.39it/s]


upload the weights file


In [57]:
model.configuration['weights_filename'] = artifact.filename
model.update()


Model(id='68108429c53a7dc4cbdb52f6', creator='husam.m@dataloop.ai', created_at='2025-04-29T07:47:53.616Z', model_artifacts=[ItemArtifact(dataset_url=None, created_at=None, updated_at=None, updated_by=None, dataset_id=None, filename=None, name=None, type='item', id='68108485ffba37247e40dd8a', spec=None, creator=None, _description=None, annotations_count=None)], name='rf-detr', description="RF-DETR is a state-of-the-art transformer-based object detection model developed by Roboflow. It combines the power of DETR (DEtection TRansformer) architecture with Roboflow's optimizations for real-time performance. The model supports detection of 80 COCO classes with high accuracy and efficient inference capabilities. It's particularly optimized for deployment in production environments with robust performance across various object detection scenarios.", labels=['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'park

In [ ]:
print(f"model status before artifact upload: {model.status}")
# Upload your weights
artifact = model.artifacts.upload(filepath='rf-detr-base-coco.pth')

# Update the configuration
model.configuration['weights_filename'] = artifact.filename


model = project.models.get(model_name = 'rf-detr')
print(f"model status after artifact upload: {model.status}")


model = project.models.get(model_name = 'rf-detr')
print(f"model status after deploy: {model.status}")

In [ ]:
model.status = 'deployed'
model.update()

# # # Deploy your model
# print("-##################\n##################\n##################\n##################\n##################\n deploy only once \n##################\n##################\n##################\n##################\n##################\n")
# model.deploy()

In [ ]:
cloned_model_taxi = project.models.get(model_name='rf-detr-tex4l')
artifact = cloned_model_taxi.artifacts.upload(filepath='model.pth')

# Update the configuration
cloned_model_taxi.configuration['weights_filename'] = artifact.filename

model.update()


In [ ]:
model = project.models.get(model_name = 'rf-detr')

dataset = project.datasets.get(dataset_name='husam_test_ds' if use_rc_env else 'husam_dogs_ds')
image_url = 'https://media.roboflow.com/notebooks/examples/dog-2.jpeg'
item_3 = dataset.items.upload(image_url)
image_url = 'https://media.roboflow.com/notebooks/examples/dog-3.jpeg'
item_4 = dataset.items.upload(image_url)
prediction_3 = model.predict(item_ids=[item_3.id,item_4.id],dataset_id=dataset.id)
prediction_3.wait()
item_3.open_in_web()

In [7]:
item_3.open_in_web()

In [6]:
model = project.models.get(model_name = 'rf-detr')
# model.open_in_web()
# project.datasets.get(dataset_name='husam_train_dataset').open_in_web()

In [7]:

# Train with data splitting and validation
train_filters = dl.Filters(field='dir', values='/train')
val_filters = dl.Filters(field='dir', values='/validation')


model = model.clone(
    model_name='rd-dert-sdk-clone-2',
    description='Husam trained model',
    dataset=project.datasets.get(dataset_name='Helios_finetune_global'),
    project_id=project.id,
    train_filter=train_filters,
    validation_filter=val_filters,
)

In [34]:
project.datasets.get(dataset_id='67fe285af136ebfb1a736f7c').recipes.list()

[Recipe(id='65b0dd120df8f550e78ad13c', creator='daniel.f@dataloop.ai', title='insects-run Default Recipe', project_ids=['99b9c307-10a9-41fc-b52e-c1c7c83ff684'], description='', metadata={'system': {'collectionTemplates': [], 'script': {'entryPoints': {'main': {'_instructions': []}}}, 'instructionDocument': {'itemId': '66cdd57d252b87c044eaa8a9', 'datasetId': '65b0dcf8b4a86b7b8ec72c60', 'name': 'IPM project - Count_all_insects.pdf'}}}, created_at='2024-01-24T09:49:06.000Z', updated_at='2024-10-01T15:22:37.453Z', updated_by='ludo.h@dataloop.ai', ui_settings={'fastClassificationBar': True, 'requireObjectId': True, 'requireParenting': True, 'ocrMode': False, 'showText': False, 'freeText': True, 'interpulationActiveOnLoad': True, 'allowInterpulationChange': True, 'allowAutomation': True, 'enablePixelSmoothing': True, 'allowCuboidRotation': True, 'allowAsymmetricCuboids': False, 'studioV2App': True, 'allowMinimumZoom': False, 'allowObjectIdAutoAssign': True, 'polylineArrowHead': False, 'polyl

In [36]:
project.models.get(model_name='rd-dert-sdk-small_dataset').dataset.switch_recipe(recipe_id='65b0dd120df8f550e78ad13c')

In [ ]:
exec = model.train()

In [ ]:
s = dl.services.get(service_id="67b4cb48e26f1d1d9699a4c4")
s.log(system=True,
          follow=True)

In [7]:
s = dl.services.get(service_id="680dfb59a8bde1d6431b590f")
s.log(follow=True)

In [6]:
trained_model = project.models.get(model_name='rf-detr-clone-large-b1')
trained_model.artifacts.list()

Iterate Pages: 100%|██████████| 1/1 [00:00<?, ?it/s]


[ItemArtifact(dataset_url='https://gate.dataloop.ai/api/v1/datasets/65b0dcf8b4a86b7b8ec72c60', created_at='2025-05-08T05:17:38.224Z', updated_at='2025-05-08T05:17:38.388Z', updated_by='husam.m@dataloop.ai', dataset_id='65b0dcf8b4a86b7b8ec72c60', filename='/artifacts/models/rf-detr-clone-large-b1/rf-detr-base-coco.pth', name='rf-detr-base-coco.pth', type='file', id='681c3e725fc64fb6db2ad1e5', spec=None, creator='husam.m@dataloop.ai', _description=None, annotations_count=0),
 ItemArtifact(dataset_url='https://gate.dataloop.ai/api/v1/datasets/65b0dcf8b4a86b7b8ec72c60', created_at='2025-05-08T12:15:12.395Z', updated_at='2025-05-08T12:15:18.909Z', updated_by='bot.2cfffec4-1b08-4698-9402-485ecdff98e4@bot.dataloop.ai', dataset_id='65b0dcf8b4a86b7b8ec72c60', filename='/artifacts/models/rf-detr-clone-large-b1/checkpoint_best_total.pth', name='checkpoint_best_total.pth', type='file', id='681ca05614c72ec875c39f72', spec=None, creator='bot.2cfffec4-1b08-4698-9402-485ecdff98e4@bot.dataloop.ai', _de

In [ ]:
trained_model.artifacts.download(artifact_name='checkpoint.pth')

Download Items:   0%|          | 0/1 [00:00<?, ?it/s]